# 🛠️ “Kendi API’mizi” Sıfırdan Yazma Yol Haritası (FastAPI/Flask YOK)

Amacımız: **hazır çatı kullanmadan**, **kendi HTTP sunucumuzu** ve üstünde **API katmanımızı** kurmak. En temelden başlayıp katman katman büyüteceğiz. En sonda dilersek bunu bir **kütüphaneye** dönüştürebilir ya da herkese açabiliriz.

---

## 0) Zemin: Ne İnşa Ediyoruz?
- **HTTP Sunucu çekirdeği** (sockets ile)
- **İstek/yanıt modeli** (Request/Response objeleri)
- **Router** (URL + HTTP metoduna göre fonksiyon çağırma)
- **Middleware** (loglama, CORS, auth gibi enine kesen kaygılar)
- **JSON gövde ayrıştırma** (request body parse)
- **Hata yönetimi** (try/except, status codes, error response)
- **Statik dosya ve basit templating** (ileride)
- **TLS (HTTPS), rate limit, auth** (ileri aşama)
- **Test, benchmark, dokümantasyon** (sürekli)

> Bu parçaları adım adım ekleyip, her aşamada “çalışıyor” kanıtlarını (kabul kriterleri) kontrol edeceğiz.

---

## Aşama 1 — **Minimal HTTP Sunucu (Hello Socket)**
**Hedef:** `socket` ile TCP dinle, tek isteği al, sabit bir HTTP 200 yanıtı döndür.

- **Kapsam:**
  - `bind/accept` ile port dinlemek
  - Ham HTTP satırları: `GET / HTTP/1.1` gibi istek başlığını okumak
  - Basit yanıt: `HTTP/1.1 200 OK` + `Content-Length` + gövde
- **Kabul Kriteri:**
  - `curl http://localhost:8080` → `Hello` gibi bir gövde alır.
  - Tarayıcıyla açınca içerik görürsün.
- **Risk/Not:** Süreli bloklanma ve tek bağlanma; performans kaygısı yok.

---

## Aşama 2 — **HTTP İsteklerini Ayrıştırma (Parser)**
**Hedef:** İstek satırını ve başlıkları doğru parse eden bir katman.

- **Kapsam:**
  - Method (`GET/POST/PUT/DELETE`), Path, Versiyon ayrıştırma
  - Başlıklar (`Host`, `Content-Length`, `Content-Type`, `Connection`, vb.)
  - Query string’i path’ten ayırıp key→value dökümü
- **Kabul Kriteri:**
  - `/health?ping=1` gibi URL’lerde query’leri doğru parse eder.
  - Eksik/bozuk isteklerde 400 döner.

---

## Aşama 3 — **Response Nesnesi ve Status Kodlar**
**Hedef:** Cevap üretimi standart olsun.

- **Kapsam:**
  - Response objesi: `status_code`, `headers`, `body`
  - Yaygın status kodlar: 200, 201, 204, 400, 404, 405, 500
  - `Content-Type: application/json` ve `text/plain` desteği
- **Kabul Kriteri:**
  - `/not-found` → 404 JSON hata cevabı
  - `/plain` → `text/plain` ile basit metin

---

## Aşama 4 — **Router (Yönlendirme)**
**Hedef:** `(method, path)` → fonksiyon eşlemesi.

- **Kapsam:**
  - Route kayıt mekanizması: `GET /health`, `POST /predict` gibi
  - Dinamik path parametreleri: `/users/{id}`
  - “405 Method Not Allowed” üretme
- **Kabul Kriteri:**
  - `/health` GET çalışır; `/health` POST 405 döner.
  - `/users/42` → `{ "id": 42 }` gibi doğru parametre çıkar.

---

## Aşama 5 — **Body Ayrıştırma (JSON/Form)**
**Hedef:** `POST/PUT` gövdelerini okuyup objeye çevirmek.

- **Kapsam:**
  - `Content-Length` kadar body okuma
  - `Content-Type: application/json` ayrıştırma
  - (İleri) `application/x-www-form-urlencoded` ve `multipart/form-data`
- **Kabul Kriteri:**
  - `/echo` endpoint’i gönderilen JSON’u geri döndürür.
  - Yanlış JSON → 400 ve açıklayıcı hata.

---

## Aşama 6 — **Middleware Mekanizması**
**Hedef:** İstek-yanıt akışına “kanca” takmak.

- **Kapsam:**
  - Zincir yapısı: `before_request` → handler → `after_request`
  - **Loglama** (istek satırı, süre)
  - **CORS** (Origin kontrolü, preflight `OPTIONS`)
  - (İleri) **Rate limiting**, **Auth**, **GZip**, **ETag**
- **Kabul Kriteri:**
  - Tüm isteklere log basılır; CORS başlıkları doğru döner.
  - `OPTIONS` → 204 ve uygun CORS header’ları.

---

## Aşama 7 — **Eşzamanlılık (Concurrency)**
**Hedef:** Birden fazla isteği kaldır.

- **Kapsam:**
  - Thread-per-connection veya ThreadPool modeli
  - (İleri) async/await mimarisi (non-blocking I/O)
- **Kabul Kriteri:**
  - Basit yük testinde aynı anda 50 istek yanıtlanır.
  - Sunucu çökmez; yanıt süreleri kabul edilebilir.

---

## Aşama 8 — **Hata Yönetimi ve İstisna Yakalama**
**Hedef:** Sunucu çökmeyecek, tutarlı hata dönecek.

- **Kapsam:**
  - Global exception handler (500 üretir)
  - “Route bulunamadı” → 404 JSON
  - “Method yok” → 405
- **Kabul Kriteri:**
  - Bilinçli hata fırlatınca şablon 500 JSON yanıtı gelir.
  - Stack trace loglanır, kullanıcıya sızmaz.

---

## Aşama 9 — **JSON Yardımcıları ve Serializer**
**Hedef:** JSON üretimi tek yerden yönetilsin.

- **Kapsam:**
  - `json_response(data, status=200)` yardımcı fonksiyonu
  - Tarih/datetime/Decimal gibi tipler için genişletilebilir serializer
- **Kabul Kriteri:**
  - `/time` tarih döndürür, düzgün JSON’lanır.

---

## Aşama 10 — **CORS ve Güvenlik Sertleştirme**
**Hedef:** Tarayıcı entegrasyonu ve temel güvenlik.

- **Kapsam:**
  - CORS allowlist (origin, methods, headers)
  - Bazı güvenlik başlıkları: `X-Content-Type-Options`, `X-Frame-Options`
- **Kabul Kriteri:**
  - Frontend’ten fetch çalışır; preflight doğru yanıtlanır.

---

## Aşama 11 — **Statik Dosya ve Templating (Opsiyonel)**
**Hedef:** Basit bir frontend ya da dokümantasyon sun.

- **Kapsam:**
  - `GET /static/*` ile dosya servisleme
  - Küçük bir şablon motoru (veya minimal string replace)
- **Kabul Kriteri:**
  - `/docs` HTML sayfası servis edilir.

---

## Aşama 12 — **TLS/HTTPS (Opsiyonel)**
**Hedef:** Şifreli iletişim.

- **Kapsam:**
  - Self-signed sertifika ile HTTPS
  - (Gerçek ortamda) ters proxy (Nginx/Caddy) arkasında çalıştırma
- **Kabul Kriteri:**
  - `https://localhost:8443` üzerinden erişim sağlanır.

---

## Aşama 13 — **Kimlik Doğrulama (Auth)**
**Hedef:** Korumalı endpoint’ler.

- **Kapsam:**
  - API key, Bearer token (JWT), Basic auth
  - Middleware ile token doğrulama
- **Kabul Kriteri:**
  - `/predict` korumalı; geçersiz token → 401

---

## Aşama 14 — **Ortam ve Yapılandırma (Config)**
**Hedef:** Port, log seviyesi, CORS gibi ayarlar dışarıdan.

- **Kapsam:**
  - `.env` veya `ENV` değişkenleri
  - Farklı profiller: `dev`, `prod`
- **Kabul Kriteri:**
  - Port/host/env değiştirilebilir.

---

## Aşama 15 — **Testler ve Benchmark**
**Hedef:** Güvenilirlik ve performans görünürlüğü.

- **Kapsam:**
  - Birim test: router, parser, middleware
  - Yük testleri: `ab`, `wrk`, `hey`
- **Kabul Kriteri:**
  - CI benzeri akışta testler yeşil.
  - Basit RPS/latency raporu.

---

## Aşama 16 — **Versiyonlama ve Dokümantasyon**
**Hedef:** Kullanıcı ve geliştirici deneyimi.

- **Kapsam:**
  - `/v1` ve `/v2` path stratejisi
  - Route tablosundan **otomatik docs** üretimi (JSON/HTML)
- **Kabul Kriteri:**
  - `/docs` JSON → HTML docs

---

## Aşama 17 — **Kütüphaneye Evrim (Opsiyonel)**
**Hedef:** Yeniden kullanılabilir paket.

- **Kapsam:**
  - Çekirdeği modülerleştirme (server, router, middleware paketleri)
  - `setup.cfg/pyproject.toml` ile paketleme
- **Kabul Kriteri:**
  - `pip install` edilebilir, basit bir “hello api” örneği ile çalışır.

---

### Yolculuk Kuralları
- **Her aşamada çalışan bir şeyler** elde edeceğiz.
- **Geridönüşümsüz ilerlemeyeceğiz**: bir aşama stabilize olmadan ilerlemeyeceğiz.
- **Basitten karmaşığa**: önce tek thread, sonra çoklu; önce JSON, sonra form-data; önce HTTP/1.1 basit, sonra keep-alive/chunked.

---

### Sıradaki Adım (Uygulama Planı)
1. **Aşama 1**: Minimal socket tabanlı HTTP 200 OK yanıtı  
2. **Aşama 2**: HTTP başlık ve path parser  
3. **Aşama 3–4**: Response nesnesi + Router  
4. **Aşama 5–6**: JSON gövde + Middleware/CORS  
5. **Aşama 7**: Eşzamanlılık (thread pool)  

> Bir sonraki adımda Aşama 1’i hayata geçirip “kabuk sunucu”yu ayağa kaldıracağız. Kod yazımı bu sırayla ilerleyecek.
